In [287]:
import numpy as np
from openbb import obb
import altair as alt
import datetime as dt
import polars as pl
import polars.selectors as cs
import datetime as dt
import infomeasure as im
np.random.seed(92183)  # For reproducible results
start_date = dt.date(2010, 1, 1)

In [288]:
wti = obb.commodity.price.spot(provider='fred', commodity='wti', start_date=start_date)
wti = pl.DataFrame(map(lambda d: d.model_dump(), wti.results)).pivot(on='symbol', index='date')
# cpi is released once a month, downcast wti to a monthly aggregate to allow analysis
wti = wti.group_by(pl.col.date.dt.strftime("%Y-%m"), maintain_order=True).agg(cs.numeric().mean())
wti.head()

date,price_DCOILWTICO
str,f64
"""2010-01""",78.325789
"""2010-02""",76.387368
"""2010-03""",81.203478
"""2010-04""",84.292857
"""2010-05""",73.7435


In [289]:
cpi = obb.economy.cpi(country='japan,united_states,france', provider='fred', start_date=start_date)
cpi = pl.DataFrame(map(lambda d: d.model_dump(), cpi.results)).pivot(on='country', index='date')
cpi = cpi.select('date', pl.exclude('date').name.suffix('_cpi'))

# do the same as for wti
cpi = cpi.group_by(pl.col.date.dt.strftime("%Y-%m"), maintain_order=True).agg(cs.numeric().mean())
cpi.head()

date,france_cpi,japan_cpi,united_states_cpi
str,f64,f64,f64
"""2010-01""",0.011019,-0.013,0.026257
"""2010-02""",0.012807,-0.011,0.021433
"""2010-03""",0.015899,-0.011,0.02314
"""2010-04""",0.016838,-0.012,0.022364
"""2010-05""",0.016492,-0.009,0.02021


In [290]:
df = wti.join(cpi, on='date', how="right").drop_nulls(subset='^.*cpi$')
print(df.shape)
df.head()

(138, 5)


price_DCOILWTICO,date,france_cpi,japan_cpi,united_states_cpi
f64,str,f64,f64,f64
78.325789,"""2010-01""",0.011019,-0.013,0.026257
76.387368,"""2010-02""",0.012807,-0.011,0.021433
81.203478,"""2010-03""",0.015899,-0.011,0.02314
84.292857,"""2010-04""",0.016838,-0.012,0.022364
73.7435,"""2010-05""",0.016492,-0.009,0.02021


In [291]:
df.describe()

statistic,price_DCOILWTICO,date,france_cpi,japan_cpi,united_states_cpi
str,f64,str,f64,f64,f64
"""count""",138.0,"""138""",138.0,138.0,138.0
"""null_count""",0.0,"""0""",0.0,0.0,0.0
"""mean""",69.076538,null,0.010602,0.003761,0.017957
"""std""",22.751063,null,0.007335,0.010305,0.009644
"""min""",16.547619,"""2010-01""",-0.003829,-0.013,-0.001995
"""25%""",49.822,null,0.004523,-0.004,0.011849
"""50%""",63.862381,null,0.010651,0.002,0.01728
"""75%""",92.021364,null,0.015949,0.007,0.022364
"""max""",109.5325,"""2021-06""",0.0251526,0.037,0.053915


In [292]:
alt.data_transformers.disable_max_rows()
#cols = cs.expand_selector(df, cs.matches("^.*cpi$") | cs.by_name("commodity_DCOILWTICO").pct_change())
df.with_columns(
    cs.matches("^.*cpi$").mul(10), cs.by_name("price_DCOILWTICO").pct_change()
).unpivot(
    index='date', on=cs.matches("^.*cpi|price_DCOILWTICO$")
).plot.line(x='date', y='value', color='variable').properties(width=1000, title="")
#df.unpivot(index='date', on=pl.col.commodity_DCOILWTICO.pct_change())#.plot.line(x='date', y='value', color='variable').properties(width=700, title="")

alt.Chart(...)

In [293]:
X = df.get_column('price_DCOILWTICO').pct_change().fill_null(0.0)
y = df.get_column('france_cpi').to_numpy().reshape(-1)
kwargs = dict(approach="ordinal", embedding_dim=3, base=2)
for lag in range(0, 12):  # maximum a year
    est = im.estimator(X, y, measure='te', prop_time=lag, **kwargs)
    # Calculate effective transfer entropy
    #print(est.result())

    # Perform statistical testing
    te = est.result() / im.entropy(y, **kwargs) # normalized TE
    stat_test = est.statistical_test(n_tests=100, method="permutation_test")
    if te > 0.1 and stat_test.p_value < .1:
        print(f'LAG: {lag}')
        effective_te = est.effective_val() / im.entropy(y, **kwargs)
        print(f"Effective TE: {effective_te:.6f} bits")
        print(f"TE: {te:.6f} bits")
        print("p_value", stat_test.p_value)
    #print(f"90% CI: {stat_test.confidence_interval(0.9)}")
    


LAG: 2
Effective TE: 0.053505 bits
TE: 0.185250 bits
p_value 0.02
LAG: 3
Effective TE: 0.037515 bits
TE: 0.190847 bits
p_value 0.01
LAG: 7
Effective TE: 0.082950 bits
TE: 0.201317 bits
p_value 0.0


### we see the 3 months and 6 months dependencies emerging 

## TODO : optimal alignement algorithm
- we want to find a vector of positive lags maximizing TE (with p_value < 0.05)
- we also want to control overfitting

we can leverage the MDL framework for this : 
 - what is the size of data D given the model ? This is TEx->y | M; i.e the the transfer entropy of X aligned with the vector of lags and y
 - what is the size of the model itself ?
    1. `alignment.rle_id().entropy()` (kudos to data engineering) will reward aligment will long blocks of consecutive values (intuitively it's good to have a lag that's constant for a long time) + this allows comparison with the "fix-lag model"
        - question here : perhaps we want to consider `alignment.pct_change().rle_id().entropy()` to avoid penalizing linearly decreasing/increaseing blocks that points to a single shock from x leading to values in y 
    2. `aligment.entropy()` will reward alignments with a lower cardinality
    - both 1. and 2. control for the model complexity
  
 - a good model is a model that maximizes TE on the data while minimizing entropies on the alignment vector
 - with this model we can trigger early stopping and quickly discard overly complex models

How do we generate candidates to be submitted to MDL ?
 - genetic algorithms with gaussian noise ?
 - bisection


## research question
is this optimal alignment, computed without accounting for confounders, sufficient to move on to causal inference which requires confounders ?